# Generating Annihilation Rates from Remote Cache of Particle Model
Tim Tyree<br>
11.23.2021

DONE: ported this ipynb to lib

In [41]:
from scipy.optimize import minimize

from lib.my_initialization import *
from lib import *
#from lib.lib_care.measure.level_sets import comp_longest_level_set_and_smooth
%load_ext autoreload
%autoreload 2

import random,scipy
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
import matplotlib as mpl #for colorbar
from scipy import stats
#DONE: hook this routine up to dask
#DONT: hook this routine up to dask_cuda
# import scanpy as sc
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
#     theme=cuxfilter.themes.dark
#     #     theme=cuxfilter.themes.rapids
else:
    sns.set_style('white')
#     theme=cuxfilter.themes.light
import warnings
warnings.simplefilter("ignore", UserWarning)
%load_ext autoreload
%autoreload 2
# import cupy as cp
np.random.seed(42)
# cp.random.seed(42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
#reset matplotlib settings
import matplotlib as mpl
sns.reset_orig()
mpl.rc_file_defaults()

In [43]:
# def comp_rmse_via_interp(x, *args):
#     '''find a best r,kappa pair for a given a,D pair for a given full model'''
#     a,D,q_values,w_values,interp=args
#     r,kappa=x
#     #interpolate nu and M using interp
#     X=np.array([[r,kappa,a,D]])
#     yhat = interp(X)
#     nu=yhat[0,0]
#     M=yhat[0,1]
#     # M=yhat[0,2]
#     what_values=M*q_values**nu
#     rmse_full=np.sqrt(np.mean((what_values-w_values)**2))
#     return rmse_full

In [44]:
# def comp_mse_via_interp(x, *args):
#     '''find a best r,kappa pair for a given a,D pair for a given full model'''
#     a,D,q_values,w_values,interp=args
#     r,kappa=x
#     #interpolate nu and M using interp
#     X=np.array([[r,kappa,a,D]])
#     yhat = interp(X)
#     nu=yhat[0,0]
#     M=yhat[0,1]
#     # M=yhat[0,2]
#     what_values=M*q_values**nu
#     mse=np.mean((what_values-w_values)**2)
#     return mse

In [45]:
wjr=recall_powerlaw_fits_to_full_models()
print(*wjr)

fk_pbc lr_pbc fk_ncbc lr_ncbc


In [46]:
#apparent measurements
# #FK (a_hat short timescale, D_hat long timescale)
a_hat_FK=7.3923;
D_hat_FK=0.365238#;num_pairs=25;tavg_step=5;tavg1_max=15;tavg2_max=15

a_hat_FK_long = 1.509282#+/-0.178827 cm^2/s, tmax=90
a_hat_FK_vlong = 1.604153#+/-0.040173 cm^2/s
# tavg2=14 ms, num_samples=19087, tmin=0, tmax=300 ms


#msr from the particle model
# npz_dir_FK='/home/timothytyree/Documents/GitHub/bgmc/python/data/control_data/arrays/control_a_7.3923_D_0.365238.npz'#csv'
#msr from the full model
# msr_dir_fk='/home/timothytyree/Documents/GitHub/bgmc/python/data/full_results/msr_fk_boostrapped_tavg2_14.0_n_19087_a_1.6_tmax_300.csv'

#LR (a_hat short timescale, D_hat long timescale)
a_hat_LR=8.595
D_hat_LR=0.586055#;num_pairs=25;tavg_step=5;tavg1_max=15;tavg2_max=15
#msr from the particle model
# npz_dir_LR='/home/timothytyree/Documents/GitHub/bgmc/python/data/control_data/arrays/control_a_8.595_D_0.586055.npz'#.csv'
a_hat_LR_long = 3.535902#+/-0.312245 cm^2/s, tmax=60

# define module

In [47]:
def comp_rmse_powerlaw(df,q_values,M,m,dq=0.05):
    """
    Example Usage:
input_fn=f"/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_24_all_powerlaw_fits.csv"
df=pd.read_csv(input_fn)
df = comp_rmse_powerlaw(df,q_values,M,m,dq=0.05)
df.sort_values(by='rmse',inplace=True)
df['rmse'].values
    """
    #compute q_values used by df
    q_min = df.q_min.drop_duplicates().values[0]
    q_max = df.q_max.drop_duplicates().values[0]
    q_values=np.arange(q_min,q_max,dq)
    #input: m,M, q_values used by df
    #output: w_values
    w_values=M*q_values**m

    #compute rmse for each row
    df['rmse']=np.nan
    for i,row in df.iterrows():
        m_ = row['m']
        M_ = row['M']
        w_values_linear=M_*q_values**m_
        df.loc[i,'rmse']= np.sqrt(np.mean((w_values_linear-w_values)**2))
    return df

# TODO(later): merge two files that cache some cloud results
then,
- TODO: investigate whether i can get good powerlaw fits with r~1.7 cm

In [29]:
#src: lst of cache files
#dst: one .csv file with lst merged


#input: df1,df2
#output: df_merged

# load the resulting table of powerlaw fits and train interpolation to estimate annihilation rates from the particle model

In [30]:
#DONE: combine the following into 'recall_particle_model_interp():'
#TODO: test ^that for figure 3
#TODO: send to lib
#TODO: test from lib for figure 2

In [31]:
#define constant parameters
reflect=0
force_code=2
set_second=0
no_attraction=0
no_repulsion=0
# kappa=100
# L=10
L=5
testing=False #True yields an extra runtimme of ~2 minutes

In [59]:
# input_fn=f"{nb_dir}/data/osg_output/run_17_all_powerlaw_fits.csv"
# input_fn=f"{nb_dir}/data/osg_output/run_18_all_powerlaw_fits.csv"
# input_fn=f"{nb_dir}/data/osg_output/run_19_all_powerlaw_fits.csv"
# input_fn=f"{nb_dir}/data/osg_output/run_20_all_powerlaw_fits.csv"#too low varkappa
# input_fn=f"{nb_dir}/data/osg_output/run_21_all_powerlaw_fits.csv"#too low varkappa
# input_fn=f"{nb_dir}/data/osg_output/run_22_all_powerlaw_fits.csv" #t/'oo low varkappa
# input_fn=f"{nb_dir}/data/osg_output/run_23_all_powerlaw_fits.csv" #too low varkappa
# input_fn=f"{nb_dir}/data/osg_output/run_24_all_powerlaw_fits.csv"
input_fn=f"{nb_dir}/data/osg_output/run_25_all_powerlaw_fits.csv"
df=pd.read_csv(input_fn)

print(f"parameter range:")
print(df.describe()[['varkappa','D','L','r','kappa']].T[['min','max']].T)
print(f"\nalinear values considered: {sorted(df['varkappa'].drop_duplicates().values)} cm^2/s")

# print(f"\n\n(more details:)")
# print(df.describe().T[['min','max']])


parameter range:
     varkappa      D    L     r  kappa
min     1.553  0.090  5.0  0.01  100.0
max     9.969  0.114  5.0  0.49  990.0

alinear values considered: [1.553, 9.969] cm^2/s


__Results:__

r=0.05-0.5 was were we spent our computational resources...  I'm rerunning a new grid search over (r,kappa) on the osg specific to the (alinear,D) values we observed from the full models.

- DONE: dev run 24 (")
- DONE: test run 24
- DONE: run run 24

In [60]:
# # r_lst=list(np.sqrt(np.arange(0.1,5,0.2)))
# # r_lst=list(np.sqrt(np.arange(0.1,1.7,0.2)))
# r_lst=list(np.arange(0.1,1.8,0.2))
# r_lst
# np.arange(0.1,1.8,0.2)

In [61]:


# #query the DataFrame
# query =(df.set_second==set_second)&(df.reflect==reflect)
# query&=(df.no_repulsion==no_repulsion)&(df.no_attraction==no_attraction)
# query&=(df.neighbor==neighbor)&(df.force_code==force_code)
# # query&=df.r==r
# # query&=df.kappa==kappa
# # query&=df.D==D
# query&=df.L==L
# # query&=df.varkappa==varkappa
# dg=df[query]

In [62]:
# #neighbor-only forces
# input_fn=f"/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_19_all_powerlaw_fits.csv"
# neighbor=1
# vector-summed forces
# input_fn=f"/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_17_all_powerlaw_fits.csv"
# input_fn=f"{nb_dir}/data/osg_output/run_17_all_powerlaw_fits.csv"
neighbor=0

#TODO: aggregate powerlaw fits with neighbor-only or vector-summed fits 
#TODO: aggregate powerlaw fits with D>=1.2


df=pd.read_csv(input_fn)
#query the DataFrame
query =(df.set_second==set_second)&(df.reflect==reflect)
query&=(df.no_repulsion==no_repulsion)&(df.no_attraction==no_attraction)
query&=(df.neighbor==neighbor)&(df.force_code==force_code)
# query&=df.r==r
# query&=df.kappa==kappa
# query&=df.D==D
query&=df.L==L
# query&=df.varkappa==varkappa
dg=df[query]

In [63]:
#DONE: recall desired powerlaw fit
# Recall powerlaw fits to full models
wjr=recall_powerlaw_fits_to_full_models()
print(*wjr)

#select fenton-karma model
dict_fit = wjr['fk_pbc']
alinear= 1.553 #(16) cm^2/s
D = 0.114 #(8) cm^2/s

# #select luo-rudy model
# dict_fit = wjr['lr_pbc']
# alinear= 9.969 #(100) cm^2/s
# D = 0.09 #(50) cm^2/s


print_dict(dict_fit)
m=dict_fit['m']
M=dict_fit['M']

fk_pbc lr_pbc fk_ncbc lr_ncbc
m=1.8772341309722325
Delta_m=0.02498750277237229
M=5.572315674840435
Delta_M=0.3053120355191732
b=1.665608066257863
Delta_b=0.029341409948945123


In [64]:
# # input_fn=f"/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_24_all_powerlaw_fits.csv"
# input_fn=f"/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_25_all_powerlaw_fits.csv"
# df=pd.read_csv(input_fn)
# # df = comp_rmse_powerlaw(df,q_values,M,m,dq=0.05)
# df = comp_rmse_powerlaw(df,M=M,m=m,dq=0.05)
# df.sort_values(by='rmse',inplace=True)
# df['rmse'].values

In [67]:
# r=1.7
col_lst=['varkappa','D','r','kappa','rmse']
# col_lst=['varkappa','D','r','kappa','rmse', 'm', 'M']
# boo = df['r']==r
print(f"{alinear=}, {D=}")
print(f"showing settings with {r=}, sorted by rmse")
# dg=df[boo]
dg=df#[boo]
# df[boo][col_lst].head(20)

alinear=1.553, D=0.114
showing settings with r=1.7, sorted by rmse


root cause confirmed...
- RESOLVED: run #25 involves ranges up to r=1.7, which was only in run #24
    - DONE: redownload run #25 from osg
            - ERROR: mv: cannot move 'run_X_all' to 'run_25_all/run_X_all': Directory not empty
            - ^that fixed it.
    - DONE: recompute powerlaw fits


- TODO: import powerlaw fits and find a low range r,kappa setting that reproduces the fit of the full model

In [68]:
#define parameters to be varied
# input_cols=['r','D','varkappa']#
input_cols=['r','kappa','varkappa','D']#
# output_cols=['m','Delta_m','M','Delta_M']#2 minute runtime
output_cols=['m','M']#2.75 minute runtime paradoxically...
#inputs:dg,input_cols,output_col
#output: fitted model

Xall=dg[input_cols].values 
yall=dg[output_cols].values

In [69]:
X=Xall.copy()
y=yall.copy()
m = len(y) # number of training examples
print(f'number of training examples is {m:d}.')
print(f"estimated runtime is ~{20+120*int(testing)} seconds...")

interp = LinearNDInterpolator(X, y)
# interp = CloughTocher2DInterpolator(X, y)
print(f"training complete!")
if testing:
    yhat = interp(X)
    print(f"shape of X {X.shape} --> shape of y=yhat {yhat.shape}")

    rmse=np.sqrt(np.mean((yhat-y)**2))
    print(f"the rmse of simple interpolation is {rmse:.4f}")

number of training examples is 17640.
estimated runtime is ~20 seconds...
training complete!


# Generate plots of annihilation rates for each apparent parameter setting

## Luo-Rudy

In [70]:
interp

In [ ]:
model_name='lr_pbc'
c='C1'
# model_name='fk_pbc'
# c='C0'

#compute annihilation rates 
nu_full=wjr[model_name]['m']
M_full=wjr[model_name]['M']
q_values=np.linspace(0,1,20)
w_values=M_full*q_values**nu_full

if model_name=='lr_pbc':
    model_name_string='Luo-Rudy'
elif model_name=='fk_pbc':
    model_name_string='Fenton-Karma'
else:
    model_name_string='??'

In [ ]:
#DONE: plotted how bad the fits are if LR gets alinear~3 instead of alinear~10
# a=a_hat_LR_long
# # a=a_hat_LR
# D=D_hat_LR

# #from fit to wj's FK positions (200x200, diffcoeff: 0.0005)
# a=1.553
# D=0.114

#from fit to wj's LR positions (200x200, diffcoeff: 0.001)
a=9.969
D=0.09

#find a best r,kappa pair for a given a,D pair for a given full model
args=a,D,q_values,w_values,interp
bnds = ((1e-3, 2), (1e2, 1e4))
# x0 = (0.1, 500)
# x0 = (.2, 100)
# x0 = (.3, 100)
x0 = (.3, 100)
res = minimize(comp_rmse_via_interp, x0, args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
# res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
mse=comp_mse_via_interp(res.x, *args)
rmse=np.sqrt(mse)
# rmse=comp_rmse_via_interp(res.x, *args)

#compute the resulting annihilation rates
r=res.x[0]
kappa=res.x[1]
X=np.array([[r,kappa,a,D]])
yhat = interp(X)
nu=yhat[0,0]
M=yhat[0,1]
# M=yhat[0,2]
what_values=M*q_values**nu

#plot the result over the original annihilation rates
title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
plt.plot(q_values,what_values,c='C4',label='Best Fit to\nParticle Model',alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
plt.legend(fontsize=15)
plt.title(title,fontsize=15)
plt.show()
print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

In [ ]:
#DONE: plotted how bad the fits are if LR gets alinear~3 instead of alinear~10
# a=a_hat_LR_long
# # a=a_hat_LR
# D=D_hat_LR

# #from fit to wj's LR positions (200x200, diffcoeff: 0.001)
# a=9.969
# D=0.09

# #find a best r,kappa pair for a given a,D pair for a given full model
# args=a,D,q_values,w_values,interp
# bnds = ((1e-3, 2), (1e2, 1e4))
# # x0 = (0.1, 500)
# # x0 = (.2, 100)
# # x0 = (.3, 100)
# x0 = (.3, 100)
# res = minimize(comp_rmse_via_interp, x0, args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
# # res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
# mse=comp_mse_via_interp(res.x, *args)
# rmse=np.sqrt(mse)
# # rmse=comp_rmse_via_interp(res.x, *args)

# #compute the resulting annihilation rates
# r=res.x[0]
# kappa=res.x[1]
# X=np.array([[r,kappa,a,D]])
# yhat = interp(X)
# nu=yhat[0,0]
# M=yhat[0,1]

# # col_lst=['varkappa','D','r','kappa','rmse', 'm', 'M']
# # 9.969	0.090	1.7	10	7.584443	2.657610	40.864289
# r = 1.7
# kappa = 10 
# nu = 2.657610
# M = 40.864289
# rmse = 7.584443
# # M=yhat[0,2]
# what_values=M*q_values**nu

# col_lst=['varkappa','D','r','kappa','rmse', 'm', 'M']
# 9.969	0.090	1.7	10	7.584443	2.657610	40.864289
r = 1.7
kappa = 10
nu = 2.657610
M = 40.864289
rmse = 7.584443
# M=yhat[0,2]
what_values=M*q_values**nu

#plot the result over the original annihilation rates
title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
plt.plot(q_values,what_values,c='C4',label='Best Fit to\nParticle Model',alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
plt.legend(fontsize=15)
plt.title(title,fontsize=15)
plt.show()
print(f"a={a}, D={D}, zr={r}, kappa={kappa}, rmse={rmse}")

In [ ]:
# a=a_hat_LR_long
a=a_hat_LR
D=D_hat_LR
#find a best r,kappa pair for a given a,D pair for a given full model
args=a,D,q_values,w_values,interp
bnds = ((1e-3, 2), (1e2, 1e4))
x0 = (0.1, 500)
res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
mse=comp_mse_via_interp(res.x, *args)
rmse=np.sqrt(mse)
# rmse=comp_rmse_via_interp(res.x, *args)


#compute the resulting annihilation rates
r=res.x[0]
kappa=res.x[1]
X=np.array([[r,kappa,a,D]])
yhat = interp(X)
nu=yhat[0,0]
M=yhat[0,1]
# M=yhat[0,2]
what_values=M*q_values**nu

#plot the result over the original annihilation rates
title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
plt.plot(q_values,what_values,c='C4',label='Best Fit to\nParticle Model',alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
plt.legend(fontsize=15)
plt.title(title,fontsize=15)
plt.show()
print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

## Fenton-Karma

In [ ]:
# model_name='lr_pbc'
# c='C1'
model_name='fk_pbc'
c='C0'

#compute annihilation rates 
nu_full=wjr[model_name]['m']
M_full=wjr[model_name]['M']
q_values=np.linspace(0,1,20)
w_values=M_full*q_values**nu_full

if model_name=='lr_pbc':
    model_name_string='Luo-Rudy'
elif model_name=='fk_pbc':
    model_name_string='Fenton-Karma'
else:
    model_name_string='??'

In [ ]:
# #DONE: plotted how bad the fits are if FK gets alinear~7 instead of alinear~1.5
# a=a_hat_FK
# D=D_hat_FK

#from fit to wj's FK positions (200x200, diffcoeff: 0.0005)
a=1.553
D=0.114

#find a best r,kappa pair for a given a,D pair for a given full model
args=a,D,q_values,w_values,interp
bnds = ((1e-3, 2), (1e2, 1e4))
# x0 = (0.1, 500)
# x0 = (0.3, 700)
# x0 = (0.5, 400)
res = minimize(comp_rmse_via_interp, x0, args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
# res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
mse=comp_mse_via_interp(res.x, *args)
rmse=np.sqrt(mse)
# rmse=comp_rmse_via_interp(res.x, *args)

#compute the resulting annihilation rates
r=res.x[0]
kappa=res.x[1]
X=np.array([[r,kappa,a,D]])
yhat = interp(X)
nu=yhat[0,0]
M=yhat[0,1]
# M=yhat[0,2]
what_values=M*q_values**nu

#plot the result over the original annihilation rates
title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
plt.plot(q_values,what_values,c='C4',label='Best Fit to\nParticle Model',alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
plt.legend(fontsize=15)
plt.title(title,fontsize=15)
plt.show()
print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

In [ ]:
.239**2*139.

In [ ]:
.3**2*100

In [ ]:
a=a_hat_FK_long
D=D_hat_FK
#find a best r,kappa pair for a given a,D pair for a given full model
args=a,D,q_values,w_values,interp
bnds = ((1e-3, 2), (1e2, 1e4))
x0 = (0.1, 500)
res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
mse=comp_mse_via_interp(res.x, *args)
rmse=np.sqrt(mse)
# rmse=comp_rmse_via_interp(res.x, *args)

#compute the resulting annihilation rates
r=res.x[0]
kappa=res.x[1]
X=np.array([[r,kappa,a,D]])
yhat = interp(X)
nu=yhat[0,0]
M=yhat[0,1]
# M=yhat[0,2]
what_values=M*q_values**nu

#plot the result over the original annihilation rates
title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
plt.plot(q_values,what_values,c='C4',label='Best Fit to\nParticle Model',alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
plt.legend(fontsize=15)
plt.title(title,fontsize=15)
plt.show()
print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

In [ ]:
a=a_hat_FK_vlong
D=D_hat_FK
#find a best r,kappa pair for a given a,D pair for a given full model
args=a,D,q_values,w_values,interp
bnds = ((1e-3, 2), (1e2, 1e4))
x0 = (0.1, 500)
res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
mse=comp_mse_via_interp(res.x, *args)
rmse=np.sqrt(mse)
# rmse=comp_rmse_via_interp(res.x, *args)

#compute the resulting annihilation rates
r=res.x[0]
kappa=res.x[1]
X=np.array([[r,kappa,a,D]])
yhat = interp(X)
nu=yhat[0,0]
M=yhat[0,1]
# M=yhat[0,2]
what_values=M*q_values**nu

#plot the result over the original annihilation rates
title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
plt.plot(q_values,what_values,c='C4',label='Best Fit to\nParticle Model',alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
plt.legend(fontsize=15)
plt.title(title,fontsize=15)
plt.show()
print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

# TODO: Generate plots of annihilation rates for a given pair of parameter settings fit to the full models

In [ ]:
# minimize

In [48]:
# def PlotParticlModelAnnihilationRateFit(model_name,c,a,D,wjr,interp,ax=None,**kwargs):
#     """
#     Example Usage:
#     interp=recall_particle_model_interp()
#     dict_out=PlotParticlModelAnnihilationRateFit(model_name,c,a,D,wjr,interp,ax=None)
#     """
#     if ax is None:
#         ax=plt.gca()
#     plt.sca(ax)
#     #compute annihilation rates 
#     nu_full=wjr[model_name]['m']
#     M_full=wjr[model_name]['M']
#     q_values=np.linspace(0,1,20)
#     w_values=M_full*q_values**nu_full

#     if model_name=='lr_pbc':
#         model_name_string='Luo-Rudy'
#     elif model_name=='fk_pbc':
#         model_name_string='Fenton-Karma'
#     else:
#         model_name_string='??'

#     #find a best r,kappa pair for a given a,D pair for a given full model
#     args=a,D,q_values,w_values,interp
#     bnds = ((1e-3, 2), (1e2, 1e4))
#     x0 = (0.1, 500)
#     res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
#     mse=comp_mse_via_interp(res.x, *args)
#     rmse=np.sqrt(mse)
# #     rmse=comp_rmse_via_interp(res.x, *args)
# #     # print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

#     #compute the resulting annihilation rates
#     r=res.x[0]
#     kappa=res.x[1]
#     X=np.array([[r,kappa,a,D]])
#     yhat = interp(X)
#     nu=yhat[0,0]
#     M=yhat[0,1]
#     # M=yhat[0,2]
#     what_values=M*q_values**nu

#     #plot the result over the original annihilation rates
#     title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
#     plt.plot(q_values,w_values,c='k',alpha=0.1,lw=3)
#     plt.plot(q_values,what_values,'--',c=c,label=f'Fit to\n{model_name_string}',alpha=1)
#     #plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
#     #plt.plot(q_values,what_values,c='C4',label='Fit to\nParticle Model',alpha=0.5)
#     format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
#     plt.legend(fontsize=15)
#     plt.title(title,fontsize=15)
#     print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")
#     return dict(r=r,kappa=kappa,nu=nu,M=M,rmse=rmse,res=res)
    
def PlotParticlModelAnnihilationFit(model_name,c,a,D,wjr,ax=None,**kwargs):
    """
    
    """
    if ax is None:
        ax=plt.gca()
    plt.sca(ax)
    #compute annihilation rates 
    nu_full=wjr[model_name]['m']
    M_full=wjr[model_name]['M']
    q_values=np.linspace(0,1,20)
    w_values=M_full*q_values**nu_full

    if model_name=='lr_pbc':
        model_name_string='Luo-Rudy'
    elif model_name=='fk_pbc':
        model_name_string='Fenton-Karma'
    else:
        model_name_string='??'

    #find a best r,kappa pair for a given a,D pair for a given full model
    args=a,D,q_values,w_values,interp
    bnds = ((1e-3, 2), (1e2, 1e4))
    x0 = (0.1, 500)
    res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
    mse=comp_mse_via_interp(res.x, *args)
    rmse=np.sqrt(mse)
#     rmse=comp_rmse_via_interp(res.x, *args)
#     # print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

    #compute the resulting annihilation rates
    r=res.x[0]
    kappa=res.x[1]
    X=np.array([[r,kappa,a,D]])
    yhat = interp(X)
    nu=yhat[0,0]
    M=yhat[0,1]
    # M=yhat[0,2]
    what_values=M*q_values**nu

    #plot the result over the original annihilation rates
    title=f'a={a:.3f}, D={D:.3f}'+r' cm$^2$/s'+f"\nr={r:.3f} cm, "+r'$\kappa$'+f'={kappa:.1f} Hz\nRMSE={rmse:.4f}\n'
    plt.plot(q_values,w_values,c='k',alpha=0.1,lw=3)
    plt.plot(q_values,what_values,'--',c=c,label=f'Fit to\n{model_name_string}',alpha=1)
    #plt.plot(q_values,w_values,c=c,label=model_name_string,alpha=0.5,lw=2)
    #plt.plot(q_values,what_values,c='C4',label='Fit to\nParticle Model',alpha=0.5)
    format_plot(ax=plt.gca(),xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w$ (Hz/cm$^2$)')
    plt.legend(fontsize=15)
    plt.title(title,fontsize=15)
    print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")
    return dict(r=r,kappa=kappa,nu=nu,M=M,rmse=rmse,res=res)

In [49]:
#visual estimates from the oscillatory model
D_osc_FK=0.8
D_osc_LR=1.2
a_osc_FK=1.6
a_osc_LR=8.5/2
a_hat_osc_FK=1.60
a_hatosc_LR=2.24

In [ ]:
model_name1='fk_pbc'
c1='C0'
model_name2='lr_pbc'
c2='C1'
# a1=a_osc_FK; D1=D_osc_FK;
# a2=a_osc_LR; D2=D_osc_LR;
a1=a_osc_FK; D1=D_osc_FK;
a2=a_osc_LR; D2=D_osc_LR;



kwargs={}
retval1= PlotParticlModelAnnihilationFit(model_name1,c1,a1,D1,wjr,ax=None,**kwargs)
retval2= PlotParticlModelAnnihilationFit(model_name2,c2,a2,D2,wjr,ax=None,**kwargs)
plt.show()

In [ ]:
# model_name,c,a,D=model_name1,c1,a1,D1
model_name,c,a,D=model_name2,c2,a2,D2

# if ax is None:
#     ax=plt.gca()
# plt.sca(ax)
#compute annihilation rates 
nu_full=wjr[model_name]['m']
M_full=wjr[model_name]['M']
q_values=np.linspace(0,1,20)
w_values=M_full*q_values**nu_full

if model_name=='lr_pbc':
    model_name_string='Luo-Rudy'
elif model_name=='fk_pbc':
    model_name_string='Fenton-Karma'
else:
    model_name_string='??'

#find a best r,kappa pair for a given a,D pair for a given full model
args=a,D,q_values,w_values,interp
bnds = ((1e-3, 2), (1e2, 1e4))
x0 = (0.1, 500)
res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='Nelder-Mead', bounds=bnds,tol=1e-3)
mse=comp_mse_via_interp(res.x, *args)
rmse=np.sqrt(mse)
#     rmse=comp_rmse_via_interp(res.x, *args)

#compute the resulting annihilation rates
r=res.x[0]
kappa=res.x[1]
X=np.array([[r,kappa,a,D]])
yhat = interp(X)
nu=yhat[0,0]
M=yhat[0,1]
# M=yhat[0,2]
what_values=M*q_values**nu


print(f"a={a}, D={D}, r={r}, kappa={kappa}, rmse={rmse}")

In [ ]:
args

In [ ]:
res = minimize(comp_rmse_via_interp, (0.1, 500), args, method='trust-krylov', bounds=bnds,tol=None)
mse=comp_mse_via_interp(res.x, *args)
print(f"mse={mse:.4f}")


__Methods Available__
- 'Nelder-Mead' :ref:`(see here) <optimize.minimize-neldermead>`
- 'Powell'      :ref:`(see here) <optimize.minimize-powell>`
- 'CG'          :ref:`(see here) <optimize.minimize-cg>`
- 'BFGS'        :ref:`(see here) <optimize.minimize-bfgs>`
- 'Newton-CG'   :ref:`(see here) <optimize.minimize-newtoncg>`
- 'L-BFGS-B'    :ref:`(see here) <optimize.minimize-lbfgsb>`
- 'TNC'         :ref:`(see here) <optimize.minimize-tnc>`
- 'COBYLA'      :ref:`(see here) <optimize.minimize-cobyla>`
- 'SLSQP'       :ref:`(see here) <optimize.minimize-slsqp>`
- 'trust-constr':ref:`(see here) <optimize.minimize-trustconstr>`
- 'dogleg'      :ref:`(see here) <optimize.minimize-dogleg>`
- 'trust-ncg'   :ref:`(see here) <optimize.minimize-trustncg>`
- 'trust-exact' :ref:`(see here) <optimize.minimize-trustexact>`
- 'trust-krylov' :ref:`(see here) <optimize.minimize-trustkrylov>`
- custom - a callable object (added in version 0.14.0),
  see below for description.

__Interacting Particle Models that could potentially explain the apparent disagreement for the Luo-Rudy model at long time scales__

Considering $C=\sqrt{4D}$,

$$
dR = \frac{-2a}{R}dt + CdW(t).
$$

The attracting inverse_powerlaw_model be changed to the fractional_powerlaw_model:

$$
dR = \frac{-2a}{R^b}dt + CdW(t)
$$

for $b=0.716(1)$ (LR) or $b=0.797(2)$ (FK).


The attracting inverse_powerlaw_model be changed to the attracting and repelling oscillatory_powerlaw_model:

$$
dR = \frac{-2a}{R}\cos(\omega t + \phi)dt + CdW(t)
$$

for $\omega=2\pi/T$ for $T=100$ ms (LR) or for $T=90$ ms (FK).  To keep the model simple, $\phi=\phi_0$ is a phase constant.

# TODO: plot the full model's annihilation rates on one plot

In [17]:
1+1

2